## Pricing engines test - Anaytical vs FFT vs Montecarlo

This notebook runs a set of diagnostic tests comparing the Analytical, FFT and Montecarlo engines. It is intentionally verbose with prints and small plots to help debugging.

### Notes:
 - Run from the repository notebooks folder. The first cell will take you to the root folder of the project so the necessary modules can be imported

In [1]:
import os

# --- Step 1: Define the target folder relative to the current folder ---
# '..' means go up one level. '..\\..' means go up two levels.
relative_path_up_two = os.path.join('..', '..') 

# --- Step 2: Navigate up two levels (this assumes the notebook's current 
#             folder is the working directory, which is typical in Jupyter)
try:
    # Change the working directory to the target path
    os.chdir(relative_path_up_two)
    
    # Print the new directory for confirmation
    print(f"Successfully changed directory up two levels to: \n{os.getcwd()}")
    
except FileNotFoundError:
    print("Error: Could not navigate up two levels. Check your starting path.")

Successfully changed directory up two levels to: 
c:\Users\ramon\OneDrive\Desktop\Python projects\OptionPricingPY


In [2]:
from src.engines import *
from src.products import *
from src.models import *
from src.greeks import *

import numpy as np


# ============================================================================
# EJEMPLO DE USO
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("EJEMPLO 1: Black-Scholes con diferentes engines")
    print("="*70)
    
    # Crear producto
    option = EuropeanOption(
        S=100,
        K=100,
        T=30,  # días
        option_type='call',
        qty=1
    )
    
    # Crear modelo
    bs_model = BlackScholesModel(sigma=0.2, r=0.05, q=0.02)
    
    # Pricing con motor analítico
    analytical_engine = AnalyticalEngine()
    price_analytical = analytical_engine.calculate_price(option, bs_model)
    print(f"\nPrecio Analítico: {price_analytical:.4f}")
    
    # Pricing con FFT
    fft_engine = FFTEngine()
    price_fft = fft_engine.calculate_price(option, bs_model)
    print(f"Precio FFT: {price_fft:.4f}")
    
    # Pricing con Monte Carlo
    mc_engine = MonteCarloEngine(n_paths=100000, seed=42)
    price_mc = mc_engine.calculate_price(option, bs_model)
    print(f"Precio Monte Carlo: {price_mc:.4f}")
    
    # Parámetros del modelo Heston
    heston_model = HestonModel(
        kappa=2.0,      # velocidad de reversión a la media
        theta=0.04,     # varianza de largo plazo
        sigma=0.3,      # volatilidad de la volatilidad
        rho=-0.7,       # correlación
        v0=0.04,        # varianza inicial
        r=0.05,
        q=0.02
    )
    
    price_heston_fft = fft_engine.calculate_price(option, heston_model)
    price_heston_mc = mc_engine.calculate_price(option, heston_model)
    
    print(f"\nPrecio Heston (FFT): {price_heston_fft:.4f}")
    print(f"Precio Heston (Monte Carlo): {price_heston_mc:.4f}")
    print(f"Diferencia: {abs(price_heston_fft - price_heston_mc):.4f}")
    
    print("\n" + "="*70)
    print("EJEMPLO 3: Comparación múltiples strikes")
    print("="*70)
    
    # Parámetros similares a tu ejemplo
    S0 = 100.0
    r = 0.01
    q = 0.35
    T = 365  # 1 año en días
    vol = 0.2
    
    # Heston con parámetros que reproducen Black-Scholes
    kappa = 0
    theta = vol**2
    sigma_h = 0
    rho = 0.2
    v0 = vol**2
    
    bs_model_test = BlackScholesModel(sigma=vol, r=r, q=q)
    heston_model_test = HestonModel(
        kappa=kappa, theta=theta, sigma=sigma_h,
        rho=rho, v0=v0, r=r, q=q
    )
    
    # Array de strikes
    strikes = np.linspace(80, 120, 20)
    
    print(f"\nStrike | Analítico BS | Monte Carlo BS | Monte Carlo Heston")
    print("-" * 65)
    
    mc_engine_test = MonteCarloEngine(n_paths=100000, n_steps=252, seed=42)
    
    for K in strikes:
        opt = EuropeanOption(S=S0, K=K, T=T, option_type='call', qty=1)
        
        price_bs_analytical = analytical_engine.calculate_price(opt, bs_model_test)
        price_bs_mc = mc_engine_test.calculate_price(opt, bs_model_test)
        price_heston_mc = mc_engine_test.calculate_price(opt, heston_model_test)
        
        print(f"{K:6.1f} | {price_bs_analytical:12.4f} | {price_bs_mc:14.4f} | {price_heston_mc:18.4f}")
    
    # Calcular error L2 
    prices_bs = np.array([analytical_engine.calculate_price(
        EuropeanOption(S=S0, K=K, T=T, option_type='call', qty=1), 
        bs_model_test) for K in strikes])
    
    prices_mc = np.array([mc_engine_test.calculate_price(
        EuropeanOption(S=S0, K=K, T=T, option_type='call', qty=1), 
        heston_model_test) for K in strikes])
    
    l2_error = np.linalg.norm(prices_bs - prices_mc)
    print(f"\nL2 norm difference: {l2_error:.6f}")

EJEMPLO 1: Black-Scholes con diferentes engines

Precio Analítico: 2.4056
Precio FFT: 2.4056
Precio Monte Carlo: 2.4113

Precio Heston (FFT): 2.3939
Precio Heston (Monte Carlo): 2.3938
Diferencia: 0.0002

EJEMPLO 3: Comparación múltiples strikes

Strike | Analítico BS | Monte Carlo BS | Monte Carlo Heston
-----------------------------------------------------------------
  80.0 |       2.5784 |         2.5921 |             2.5725
  82.1 |       2.1054 |         2.1165 |             2.1015
  84.2 |       1.7089 |         1.7175 |             1.7068
  86.3 |       1.3793 |         1.3854 |             1.3781
  88.4 |       1.1073 |         1.1113 |             1.1049
  90.5 |       0.8844 |         0.8870 |             0.8806
  92.6 |       0.7030 |         0.7048 |             0.6995
  94.7 |       0.5563 |         0.5572 |             0.5531
  96.8 |       0.4384 |         0.4387 |             0.4363
  98.9 |       0.3440 |         0.3435 |             0.3426
 101.1 |       0.2690 |    